In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# pip install seaborn
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Fantasy Premier League Data Set
### Authors: John Foley, Conor Glynn and Eoghan O'Mahony.
Here is an overview of the dataset.
<br> <br>
The type of data that is included in the dataset is data on players, matches and the fantasy points from the 2016/2017 season until the 2022/2023 season. We decided that the target of our data would be to try to figure out the value or the price change of a player, from one week to the next. The total size of the dataset that we will be looking at is 117Mb. The sampling frequency is every week which has a match being played.
<br> <br>
The whole dataset is available on GitHub by this link: https://github.com/vaastav/Fantasy-Premier-League

### Goal of Our Analysis
The main goal that we want to achieve is that we want to perform a regression calculation on all of the players’ values. We want to be able to use training data to make an accurate model so that it could take the testing data and it could give a correct answer. 

As we can see here, we have a correlation matrix. This includes all of the features that are in the csv file. Our target is the value column. We can see in this matrix that there are a few columns that are fairly correlated to it; such as selected and creativity for example.

One important thing to note is the phrase: "correlation doesn't equal causation". In this way, the correlation of two or more features doesn’t necessarily mean that one is caused by the other. However, it is still very useful to look at the correlation matrix.

### Exploratory Analysis of Our Dataset
Here are some graphs that we will use to explore our dataset. The x-axis represents the gameweek and the y-axis represents the value that a certain feature had in that gameweek (depending on the line in question). These lines represent:
* Value in blue, 
* Points Gained in orange,
* Transfers In in the given gameweek in green,
* Transfers Out in the given gameweek in red.

As we can see there are some very interesting things happening to these features. We can see for example that in the week Salah got a lot of points, he was transferred in a great deal the next week. We can also see that sometimes he didn’t have his best week and so in the following week, he wasn’t transferred in as much. He must have had a very bad week in week 8 because he was transferred out a very high amount the following week. With our target, value, we can see that his value is fluctuating, based on the other factors that is going on in that gameweek.

The diagrams below examples of two pair plots that we created. These plots basically show the distribution of players for a certain feature. The x-axis is the cumulative points in plot 1 and is value in the other. The y-axis is the number of players that had these cumulative points/value. There are a lot of players that have low cumulative points/low value and as we go to the right of each graph we can see that there are very few players with high cumulative points/high value. We can see that these two graphs have very similar distributions which suggests to us that they may be highly correlated. This is interesting to us as the value is our target.

### Data Cleaning
The first step of the data cleaning is to find out the amount of null values that is in our dataframe. An easy way that we can do this is by printing the number of null values that are present in our dataframe. If we get an answer of 0 we will know that our dataframe has zero NaN values.

In [2]:
dfClean1 = pd.read_csv('data/cleaned_merged_seasons.csv')
print('We have {} NaN/Null values in this dataset'.format(dfClean1.isnull().values.sum()))

We have 49570 NaN/Null values in this dataset


C:\Users\22352155\AppData\Local\Temp\ipykernel_20564\1094262755.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dfClean1 = pd.read_csv('data/cleaned_merged_seasons.csv')


The next thing that we can do is we can use "df_Name.info()". This will show us all of the columns in the dataframe and the number of non-null values each one has. With this, we can easily see which columns we need to work on. We generate this result:

In [3]:
pd.set_option("display.max_columns", None)
dfClean1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   name               98402 non-null  object 
 3   position           98402 non-null  object 
 4   team_x             48930 non-null  object 
 5   assists            98402 non-null  int64  
 6   bonus              98402 non-null  int64  
 7   bps                98402 non-null  int64  
 8   clean_sheets       98402 non-null  int64  
 9   creativity         98402 non-null  float64
 10  element            98402 non-null  int64  
 11  fixture            98402 non-null  int64  
 12  goals_conceded     98402 non-null  int64  
 13  goals_scored       98402 non-null  int64  
 14  ict_index          98402 non-null  float64
 15  influence          98402 non-null  float64
 16  kickoff_time       984

After viewing all of the columns fully, we can see that there are 3 columns that contain null values. These are 'team_x', 'team_a_score' and 'team_h_score'. 'team_x' represents the team that the player is playing for in that gameweek, 'team_a_score' is the number of goals that the away team scored and 'team_h_score' is the number of goals that the home team scored.

Next we can look at the shape of the dataframe and get additional information about it with the following.

In [4]:
dfClean1.shape

(98402, 38)

We can now subtract the total number of non-null values that should be in any column with the actual number of non-null values that we have in the column labelled 'team_x'. With this, we can see the amount of items in this column that has null values.

In [5]:
print(98402-48930)

49472


As we can see, there are 49,472 NaN values in the 'team_x' column. The reason for this is that this column was only added in later seasons and so it didn't exist for earlier seasons. There are far too many missing for us to possibly recover and so we will remove this column.

In [6]:
#Taking away 'team_x' column
featuresGood = list(
    set(dfClean1.columns)
    - {
        "team_x",
    }
)

dfClean2 =dfClean1[featuresGood]

#info
dfClean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   minutes            98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   assists            98402 non-null  int64  
 3   position           98402 non-null  object 
 4   penalties_missed   98402 non-null  int64  
 5   transfers_in       98402 non-null  int64  
 6   penalties_saved    98402 non-null  int64  
 7   creativity         98402 non-null  float64
 8   yellow_cards       98402 non-null  int64  
 9   name               98402 non-null  object 
 10  fixture            98402 non-null  int64  
 11  ict_index          98402 non-null  float64
 12  threat             98402 non-null  float64
 13  transfers_out      98402 non-null  int64  
 14  was_home           98402 non-null  bool   
 15  round              98402 non-null  int64  
 16  kickoff_time       984

We can now see that the 'team_x' column has been removed.

There are 98 missing values between the two columns: 'team_a_score' and 'team_h_score'. To deal with these we will  now set them to 0. The reason that we chose to do this is that there are so little values that are missing (49 out of 98,402 or only about 0.05% of the data in each of these columns) and so it won't affect the dataframe in any way. A likely reason for this is that 3 or 4 games weren’t recorded correctly into the dataframe. Given that there are 380 games total in a premier league season and there are 6 years recorded, it isn’t too bad that there are only about 4 game's values missing.

In [7]:
dfClean2['team_a_score'] = dfClean2['team_a_score'].fillna(0)
dfClean2['team_h_score'] = dfClean2['team_h_score'].fillna(0)
dfClean2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98402 entries, 0 to 98401
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   minutes            98402 non-null  int64  
 1   season_x           98402 non-null  object 
 2   assists            98402 non-null  int64  
 3   position           98402 non-null  object 
 4   penalties_missed   98402 non-null  int64  
 5   transfers_in       98402 non-null  int64  
 6   penalties_saved    98402 non-null  int64  
 7   creativity         98402 non-null  float64
 8   yellow_cards       98402 non-null  int64  
 9   name               98402 non-null  object 
 10  fixture            98402 non-null  int64  
 11  ict_index          98402 non-null  float64
 12  threat             98402 non-null  float64
 13  transfers_out      98402 non-null  int64  
 14  was_home           98402 non-null  bool   
 15  round              98402 non-null  int64  
 16  kickoff_time       984

C:\Users\22352155\AppData\Local\Temp\ipykernel_20564\656274839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfClean2['team_a_score'] = dfClean2['team_a_score'].fillna(0)
C:\Users\22352155\AppData\Local\Temp\ipykernel_20564\656274839.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfClean2['team_h_score'] = dfClean2['team_h_score'].fillna(0)
